In this Data you’re challenged to analyze a Google Merchandise Store (also known as GStore, where Google swag is sold) customer dataset to predict revenue per customer. Hopefully, the outcome will be more actionable operational changes and a better use of marketing budgets for those companies who choose to use data analysis on top of GA data. For each fullVisitorId in the test set, you must predict the natural log of their total revenue. Submissions are scored on the root mean squared error (RMSE).

This kernel contains an EDA, a Baseline LightGBM model, and also Screenshots of the app. Since I am familiar with Google Analytics and also have the Google Mechandising store setup as a property in my personal environment, I though it would be good to start with some screen shots of the actual application.

# 1. Loading libraries and data

In [ ]:
library(tidyverse)
library(jsonlite)
library(scales)
library(lubridate)
library(repr)
library(ggrepel)
library(gridExtra)
library(lightgbm)

In [ ]:
train <- read_csv("../input/train.csv")
test <- read_csv("../input/test.csv")

I am using ML's script to convert the JSON columns in the train and test sets into tidy variables to make a quick start, Thanks ML (and I upvoted your kernel of course)!

In [ ]:
#JSON columns are "device", "geoNetwork", "totals", "trafficSource"

tr_device <- paste("[", paste(train$device, collapse = ","), "]") %>% fromJSON(flatten = T)
tr_geoNetwork <- paste("[", paste(train$geoNetwork, collapse = ","), "]") %>% fromJSON(flatten = T)
tr_totals <- paste("[", paste(train$totals, collapse = ","), "]") %>% fromJSON(flatten = T)
tr_trafficSource <- paste("[", paste(train$trafficSource, collapse = ","), "]") %>% fromJSON(flatten = T)

te_device <- paste("[", paste(test$device, collapse = ","), "]") %>% fromJSON(flatten = T)
te_geoNetwork <- paste("[", paste(test$geoNetwork, collapse = ","), "]") %>% fromJSON(flatten = T)
te_totals <- paste("[", paste(test$totals, collapse = ","), "]") %>% fromJSON(flatten = T)
te_trafficSource <- paste("[", paste(test$trafficSource, collapse = ","), "]") %>% fromJSON(flatten = T)

#Combine to make the full training and test sets
train <- train %>%
    cbind(tr_device, tr_geoNetwork, tr_totals, tr_trafficSource) %>%
    select(-device, -geoNetwork, -totals, -trafficSource)
    
test <- test %>%
    cbind(te_device, te_geoNetwork, te_totals, te_trafficSource) %>%
    select(-device, -geoNetwork, -totals, -trafficSource)

#Remove temporary tr_ and te_ sets
rm(tr_device)
rm(tr_geoNetwork)
rm(tr_totals)
rm(tr_trafficSource)
rm(te_device)
rm(te_geoNetwork)
rm(te_totals)
rm(te_trafficSource)

In [ ]:
#already converting some character variables into factors
factorVars <- c("channelGrouping", "browser", "operatingSystem", "deviceCategory", "country")
train[, factorVars] <- lapply(train[, factorVars], as.factor)
test[, factorVars] <- lapply(test[, factorVars], as.factor)

#also converting the data variable to the date format
train$date <- ymd(train$date)
test$date <- ymd(test$date)

#converting character variables into numeric
numVars <- c("visits", "hits", "bounces", "pageviews", "newVisits")
train[, numVars] <- lapply(train[, numVars], as.integer)
train$transactionRevenue <- as.numeric(train$transactionRevenue)
test[, numVars] <- lapply(test[, numVars], as.integer)

#converting visit start times to POSIXct
train$visitStartTime <- as.POSIXct(train$visitStartTime, tz="UTC", origin='1970-01-01')
test$visitStartTime <- as.POSIXct(test$visitStartTime, tz="UTC", origin='1970-01-01')

# 2. EDA

Below, you can see that we have over 900,000 observations in the train set. Each observation is in fact one visit to Gstore.

In [ ]:
glimpse(train)

SessionId should be the unique identifier for each observation as it is a combination of fullVisitorId and visitId (which is basically the visitStartTime before I converted it to POSIXct).  The number of unique sessionId's comes close to the total number of observations (903,653), but is 898 short.

In [ ]:
n_distinct(train$sessionId)

Let's see how we fare if I look at the sessionId's that are duplicate (n=2). As you can see this explains it perfectly! We have 898 sessionId's that appear exactly twice in the train set (898*2=1796). To do: see if I need to remove duplicate sessions, or possibly consolidate some numbers (number of pagehits, session duration etc.)

In [ ]:
train %>% count(sessionId) %>% filter(n==2) %>% summarize(sum(n))

The test set has over 800,000 observations with two variables less (transactionRevenue and campaignCode).

In [ ]:
dim(test)
setdiff(names(train), names(test))

## 2.1 Missing data

Altogether 14 variables in the train set contain missing values. Most of these variables seem related to adWords campaigns. The missing transactionRevenues must mean that there was no transaction.

In [ ]:
options(repr.plot.height=4)
NAcol <- which(colSums(is.na(train)) > 0)
NAcount <- sort(colSums(sapply(train[NAcol], is.na)), decreasing = TRUE)
NADF <- data.frame(variable=names(NAcount), missing=NAcount)
NADF$PctMissing <- round(((NADF$missing/nrow(train))*100),1)
NADF %>%
    ggplot(aes(x=reorder(variable, PctMissing), y=PctMissing)) +
    geom_bar(stat='identity', fill='blue') + coord_flip(y=c(0,110)) +
    labs(x="", y="Percent missing") +
    geom_text(aes(label=paste0(NADF$PctMissing, "%"), hjust=-0.1))

## 2.2 The response variable; transactionRevenue

Only 11, 515 sessions in the train set led to a transaction. This is only 1.27% of the observations, and that of course matches with the 98.7% missing values in the previous section. Altogether, GStore had 1,5 million USD revenues in that year.

In [ ]:
#setting missing values to zero
train$transactionRevenue[is.na(train$transactionRevenue)] <- 0

#correcting transaction revenues (see https://www.kaggle.com/c/google-analytics-customer-revenue-prediction/discussion/65775). This is for EDA purposes only. In the competition, we need to predict the log of the tranaction values as they are stored in the dataset (so not divided by 1,000,000).
y <- train$transactionRevenue #saving original values in a vector
train$transactionRevenue <- train$transactionRevenue/1000000


train %>% filter(transactionRevenue >0) %>% summarize('number of transactions'=n(), 'total revenues train set'=sum(transactionRevenue))

The range of revenues per transaction is from 1 dollarcent to 23,129.5 USD. Of the 11,515 transactions, 195 had revenues of at least 1,000 USD. The sum of revenues of this tail of big transactions is about 418,000 USD.

In [ ]:
range(train %>% select(transactionRevenue) %>% filter(transactionRevenue !=0))
train %>% filter(transactionRevenue>=1000) %>% summarize('number of transactions with at least 1000 USD revenues'=n(), 'sum revenues of transactions with at least 1000 USD revenues'=sum(transactionRevenue))


As the distribution of revenues is very right skewed, with the tail reaching 23,000 USD, I am below only displaying the histogram of the transaction with revenues below 1,000 USD.

In [ ]:
options(repr.plot.height=4)
train %>% filter(transactionRevenue >0 & transactionRevenue<1000) %>%
ggplot(aes(x=transactionRevenue)) +
        geom_histogram(fill="blue", binwidth=10) +
        scale_x_continuous(breaks= seq(0, 1000, by=100), labels = comma)

## 2.3 Time series and grouping by workday and by month
We have one year of train data (August 1, 2016-August 1, 2017), and 9 month of test data (August 2, 2017-April 30 2018).

In [ ]:
range(train$date)
range(test$date)

### 2.3.1 Time series of sessions and revenues by Date
The number of daily sessions peaked in October and November 2016, but this did not lead to higher daily revenues. In the revenues plot,  we see daily revenues vary between 0 and about 27,000 USD and that there is no real revenues trend visible. In addition, we see some outliers with high daily revenues. For instance, the high daily revenue somewhere in the beginning of April 2017. Was there a special reason for this? Maybe there was a special event, or one very big transaction?

In [ ]:
options(repr.plot.height=6)
d1 <- train %>% group_by(date) %>% summarise(dailySessions = n()) %>%
    ggplot(aes(x=date, y=dailySessions)) + geom_line(col='blue') +
    scale_y_continuous(labels=comma) + geom_smooth(col='red') +
    labs(x="", y="Sessions per Day") + scale_x_date(date_breaks = "1 month", date_labels = "%b %d")
d2 <- train %>% group_by(date) %>% summarise(dailyRevenue = sum(transactionRevenue)) %>%
    ggplot(aes(x=date, y=dailyRevenue)) + geom_line(col='blue') +
    scale_y_continuous(labels=comma) + geom_smooth(col='red') +
    labs(x="", y="Daily Revenues (USD)") + scale_x_date(date_breaks = "1 month", date_labels = "%b %d")
grid.arrange(d1,d2)

### 2.3.2 Sessions and revenues by Workday
First of all, I am creating a new variable from the Date variable with the `wday` function of lubridate.

In [ ]:
train$weekday <- wday(train$date, label=TRUE)
test$weekday <- wday(test$date, label=TRUE)
str(train$weekday)

The next thing that I am doing is that I create two functions that allow me to create barplots for all factor variables quickly.

In [ ]:
#Note: I have not included reordering of x in this function. First of all, I don't want to reorder the workday and month plots.
#Second: Manual reordering gave me the opportunity to order the Revenues in the same order as first plot (descreasing sessions). See for instance Channel Grouping.
#The adjusted functions to display plots with flipped x and y (see section 2.4.2 the source/medium dimension) includes reordering.

plotSessions <- function(dataframe, factorVariable, topN=10) {
    var_col <- enquo(factorVariable)
    dataframe %>% count(!!var_col) %>% top_n(topN, wt=n) %>%
    ggplot(aes_(x=var_col, y=~n, fill=var_col)) +
    geom_bar(stat='identity')+
    scale_y_continuous(labels=comma)+
    labs(x="", y="number of sessions")+
    theme(legend.position="none")
    }

#also creating a function to plot transactionRevenue for a factorvariable
plotRevenue <- function(dataframe, factorVariable, topN=10) {
    var_col <- enquo(factorVariable)
    dataframe %>% group_by(!!var_col) %>% summarize(rev=sum(transactionRevenue)) %>% filter(rev>0) %>% top_n(topN, wt=rev) %>% ungroup() %>%
    ggplot(aes_(x=var_col, y=~rev, fill=var_col)) +
    geom_bar(stat='identity')+
    scale_y_continuous(labels=comma)+
    labs(x="", y="Revenues (USD)")+
    theme(legend.position="none")
    }

Initially, I was a bit surprised to see that there are less sessions and revenues in weekend. However, a logical explanation to me seems that Google Swag is mostly bought by businesses rather than private individuals!

Main takeaway for modeling: The point here is that weekend sessions led to less revenues in general (numbers of sessions on Saterday/Sunday are smaller than those on weekdays, but revenues are much smaller).

In [ ]:
options(repr.plot.height=4)
w1 <- plotSessions(train, weekday)
w2 <- plotRevenue(train, weekday)
grid.arrange(w1, w2)

### 2.3.3 Sessions and revenues by Month
Below, I am repeating the trick from the previous session for `month`. Patterns are less clear to me, but months seems to matter. For instance, April has a high ratio of revenues/sessions, and this ratio is low for November.

In [ ]:
options(repr.plot.height=4)

train$month <- month(train$date, label=TRUE)
test$month <- month(test$date, label=TRUE)


m1 <- plotSessions(train, month, 12)
m2 <- plotRevenue(train, month, 12)
grid.arrange(m1, m2)

## 2.4 Channel grouping and the source/medium dimension

### 2.4.1 Channel grouping
Channels define how users come to your website (Channel Groupings are rule-based groupings of your traffic sources). The default channel groupings can be found [here](https://support.google.com/analytics/answer/3297892). These groups are mostly defined by the `Medium`, and some also by  `Source`, `Social Source Referral` or `Ad Distribution Network`. From Google Analytics:

* Source: the origin of your traffic, such as a search engine (for example, google) or a domain (example.com).
* Medium: the general category of the source, for example, organic search (organic), cost-per-click paid search (cpc), web referral (referral).

Examples:
* Organic Search is defined by the medium only (Medium exactly matches organic).
* Referral is also defined by the medium only (Medium exactly matches referral).
* Social: Social Source Referral exactly matches Yes OR Medium matches regex ^(social|social-network|social-media|sm|social network|social media)$
* Direct: Source exactly matches direct AND Medium exactly matches (not set) OR Medium exactly matches (none)

As you see in the plots, Organic Search and Social led to many sessions, but especially Social delivers hardly any revenues. Referral on the other hand delivers most revenues with a relatively small number of sessions.

In [ ]:
options(repr.plot.height=6)

#adding reordering of x manually
sessionOrder <- train %>% count(channelGrouping) %>% top_n(10, wt=n) %>% arrange(desc(n))
sessionOrder <- sessionOrder$channelGrouping

c1 <- plotSessions(train, channelGrouping) + scale_x_discrete(limits=sessionOrder)
c2 <- plotRevenue(train, channelGrouping) + scale_x_discrete(limits=sessionOrder)
grid.arrange(c1, c2)

### 2.4.2 The source/medium dimension
Source/Medium is a dimension that combines the dimensions Source and Medium. Examples of Source/Medium include google/organic, example.com/referral, and newsletter9-2014/email.

As I now wanted to flip the axis, I adjusted the plot functions first in the code below.

In [ ]:
#Note: I am using a bit of a hack to add reordering to these functions as reorder works with the normal aes() but does seem to work with aes_q()

plotSessionsFlip <- function(dataframe, factorVariable, topN=10) {
    var_col <- enquo(factorVariable)
    x <- dataframe %>% count(!!var_col) %>% top_n(topN, wt=n) %>% arrange(n)
    y <- x[[1]]
    x %>% ggplot(aes_(x=var_col, y=~n)) + coord_flip() +
    geom_bar(stat='identity', fill="orange")+
    scale_y_continuous(labels=comma)+
    labs(x="", y="number of sessions")+
    theme(legend.position="none") +
    scale_x_discrete(limits=y)
    }

plotRevenueFlip <- function(dataframe, factorVariable, topN=10) {
    var_col <- enquo(factorVariable)
    x <- dataframe %>% group_by(!!var_col) %>% summarize(rev=sum(transactionRevenue)) %>% filter(rev>0) %>% top_n(topN, wt=rev) %>% arrange(rev) %>% ungroup()
    y <- x[[1]]
    x %>% ggplot(aes_(x=var_col, y=~rev)) + coord_flip() +
    geom_bar(stat='identity', fill="orange")+
    scale_y_continuous(labels=comma)+
    labs(x="", y="Revenues (USD)")+
    theme(legend.position="none") +
    scale_x_discrete(limits=y)
    }

Below, I am first creating this dimension. It is very interesting to see that for instance "youtube.com/referral" generates a lot of traffic, but that this combi is not in the Top20 of most revenues. On the other hand, "mall.googleplex.com/referral" generates a modest amount of sessions but it is the source/medium with most revenues!

In [ ]:
train$sourceMedium <- paste(train$source, train$medium, sep="/")
test$sourceMedium <- paste(test$source, test$medium, sep="/")

s1 <- plotSessionsFlip(train, sourceMedium, 20) + theme(axis.text.x = element_text(angle = 45, hjust = 1))
s2 <- plotRevenueFlip(train, sourceMedium, 20) + theme(axis.text.x = element_text(angle = 45, hjust = 1))
grid.arrange(s1, s2, nrow=1)

## 2.5 Operating system, browser and device category

### 2.5.1 Device category
The main takeaway here is that mobile and tablet have a lot fewer sessions, and relatively low revenues per session when compared to desktop.

In [ ]:
options(repr.plot.height=5)
d1 <- plotSessions(train, deviceCategory)
d2 <- plotRevenue(train, deviceCategory)
grid.arrange(d1, d2)

### 2.5.2 Operating System
Since there are only 7 Operating Systems with revenues, I am only displaying those. Actually, (not set) had slightly more sessions than Windows Phone, but since (not set) is pretty meaningless and (not set) had no revenues anyway, I excluded (not set) from the first plot.

What stands out is Macintosh had much higher revenues than Windows with fewer sessions. In addition, it also confirms again that the mobile sessions (Android and iOS) have little revenues compared to the number of sessions.

In [ ]:
options(repr.plot.height=5)

sessionOrder <- train %>% filter(operatingSystem != "(not set)") %>% count(operatingSystem) %>% top_n(7, wt=n) %>% arrange(desc(n))
sessionOrder <- sessionOrder$operatingSystem

o1 <- plotSessions(train %>% filter(operatingSystem != "(not set)"), operatingSystem, 7) + scale_x_discrete(limits=sessionOrder)
o2 <- plotRevenue(train, operatingSystem) + scale_x_discrete(limits=sessionOrder)
grid.arrange(o1, o2)

### 2.5.3 Browser
While the first plot displays the top10 browsers with most sessions (n=10 is the default in the function), the second plot only displays 9 browsers as there were only 9 browsers with revenues. By far the most sessions and revenues come from the Chrome browser. Firefox also has a healthy balance between sessions and revenue. Safari has a lot of sessions with relatively low revenues.

In [ ]:
options(repr.plot.height=5)

d1 <- plotSessionsFlip(train, browser)
d2 <- plotRevenueFlip(train, browser)
grid.arrange(d1, d2)

## 2.6 Pageviews, Bounces and Hits

### 2.6.1 Pageviews; all sessions.

>A pageview is each time a visitor views a page on your website, regardless of how many hits are generated.

The distribution of pageviews is very right skewed. However, the second graph shows that most revenues come from (small) numbers of sessions with lots of pageviews!

In [ ]:
options(repr.plot.height=4)
#sessions with more than 28 pageviews all have frequencies of less than 1,000. Since these are hardly visible, I am excluding them.
#excluding 100 pageview NAs

p1 <- train %>% filter(!is.na(train$pageviews) & pageviews <=28) %>% 
ggplot(aes(x=pageviews)) +
    geom_histogram(fill='blue', binwidth=1) +
    scale_y_continuous(breaks=seq(0, 500000, by=100000), label=comma) +
    scale_x_continuous(breaks=seq(0, 28, by=5)) +
    coord_cartesian(x=c(0,28))

p2 <- train %>% filter(!is.na(train$pageviews) & pageviews <=28) %>% group_by(pageviews) %>%
    ggplot(aes(x=pageviews, y=transactionRevenue)) +
    geom_bar(stat='summary', fun.y = "sum", fill='blue') +
    scale_x_continuous(breaks=seq(0, 28, by=5)) +
    coord_cartesian(x=c(0,28)) + labs(y="sum of revenues")
grid.arrange(p1, p2)

Below, I am taking the mean of the revenues per session for each pageview 'category'. For the mean, we are getting some decent numbers. Not surprsingly, the medain is 0 for all those pageview categories.

In [ ]:
p1 <- train %>% filter(!is.na(train$pageviews) & pageviews <=28) %>% group_by(pageviews) %>%
    ggplot(aes(x=pageviews, y=transactionRevenue)) +
    geom_bar(stat='summary', fun.y = "mean", fill='blue') +
    scale_x_continuous(breaks=seq(0, 28, by=5)) +
    coord_cartesian(x=c(0,28)) + labs(y="mean of revenues")
p2 <- train %>% filter(!is.na(train$pageviews) & pageviews <=28) %>% group_by(pageviews) %>%
    ggplot(aes(x=pageviews, y=transactionRevenue)) +
    geom_bar(stat='summary', fun.y = "median", fill='blue') +
    scale_x_continuous(breaks=seq(0, 28, by=5)) +
    coord_cartesian(x=c(0,28)) + labs(y="median of revenues")
grid.arrange(p1, p2)

### 2.6.2 Pageviews; only sessions with any transaction revenue.
Below I displaying the histogram of **pageviews with revenues**. This mean that we are only looking at the 11,515 sessions with revenues. This paints a very different picture!

In [ ]:
train %>% filter(!is.na(train$pageviews) & pageviews <=28 & transactionRevenue>0) %>% 
ggplot(aes(x=pageviews)) +
    geom_histogram(fill='blue', binwidth=1) +
    scale_x_continuous(breaks=seq(0, 28, by=5)) +
    coord_cartesian(x=c(0,28)) +
    labs(y='number of session with transaction revenue')

I am now adding transactionRevenue>0 to the filter, and display the mean and median revenues of the sessions with some transaction revenue only. It would say that let's say the mean and median of the categories up-to 7 pageviews are very unreliable as these are based on only 0-25 transactions (of large numbers of sessions).

In [ ]:
p1 <- train %>% filter(!is.na(train$pageviews) & pageviews <=28 & transactionRevenue>0) %>% group_by(pageviews) %>%
    ggplot(aes(x=pageviews, y=transactionRevenue)) +
    geom_bar(stat='summary', fun.y = "mean", fill='blue') +
    scale_x_continuous(breaks=seq(0, 28, by=5)) +
    coord_cartesian(x=c(0,28)) + labs(y="mean of revenues") +
    geom_label(stat = "count", aes(label = ..count..), y=0, size=2)
p2 <- train %>% filter(!is.na(train$pageviews) & pageviews <=28 & transactionRevenue>0) %>% group_by(pageviews) %>%
    ggplot(aes(x=pageviews, y=transactionRevenue)) +
    geom_bar(stat='summary', fun.y = "median", fill='blue') +
    scale_x_continuous(breaks=seq(0, 28, by=5)) +
    coord_cartesian(x=c(0,28)) + labs(y="median of revenues") +
    geom_label(stat = "count", aes(label = ..count..), y=0, size=2)
grid.arrange(p1, p2)

### 2.6.3 Pageviews versus percent transactions
After having made all these revenue-related plots, a last thing that I wanted to figure out was the percent of transaction (regardless of the transaction revenue amount) for each category of pageviews. All sessions with less than 7 pageviews led to zero or a negligable percent of transactions! As you can see, there seems to be a correlation but only among pageviews of let's say at least 7-40/50 pageviews. Above 40/50 pageviews the percent transactions remain flat.

In [ ]:
train$transaction <- ifelse(train$transactionRevenue > 0, 1, 0)

train %>% filter(!is.na(train$pageviews) & pageviews <=100) %>% group_by(pageviews) %>%
summarize('sessions'= n(), 'transactions'= sum(transaction), 'pctTransactions'=round(x=((transactions/sessions)*100),digits=1)) %>%
    ggplot(aes(x=pageviews, y=pctTransactions)) +
    geom_bar(stat='identity', fill='blue') +
    scale_x_continuous(breaks=seq(0, 100, by=5)) +
    geom_smooth()

### 2.6.4 Bounces

>A bounce is a single-page session on your site.

Since a bounce is the same as a 'one pageview session'. Therefore, I am just checking if these numbers match.

In [ ]:
#setting missing values to zero (percent missing in train is 50.1%)
train$bounces[is.na(train$bounces)] <- 0L
test$bounces[is.na(test$bounces)] <- 0L
train %>% filter(bounces==1) %>% summarize('number of bounces'=n(), 'revenues bounce sessions'=sum(transactionRevenue))

So we have 450,630 bounces. However, below you can see that we have 452,522 sessions with 1 pageview. This number should be the same! 

As a bounce is a 1-page session, it should never have any revenues (A transaction needs at least 2 pageviews: first you would have to select a product, then move on to a page on which you can pay). Above, you can see that this is correct for all sessions that are labeled as a bounce. However, you can see that the zero revenues are also correct for the 452,522 sessions with pageviews==1. Therefore, I think we can fix this discrepancy with setting all those 1-page session to bounce==1.

In [ ]:
train %>% filter(pageviews==1) %>% summarize('number of 1-page sessions'=n(), 'revenues 1-page session'=sum(transactionRevenue))

### 2.6.5 Hits
The first definition that I found was:

>Hit: Pages are comprised of files. For example, if you have a page with 10 pictures, then a request to a server to view that page generates 11 hits (10 for the pictures, and one for the html file).

That did not sound as a reliable predictor to me. However, there was an interesting finding in this discussion topic: [Finding - Pageviews/Hits ratio <0.4 guarantees no revenue](https://www.kaggle.com/c/ga-customer-revenue-prediction/discussion/66774). I was pointed to another definition:

>Hit: An interaction that results in data being sent to Analytics. Common hit types include page tracking hits, event tracking hits, and ecommerce hits. Each time the tracking code is triggered by a user’s behavior (for example, user loads a page on a website or a screen in a mobile app), Analytics records that activity. Each interaction is packaged into a hit and sent to Google’s servers. Examples of hit types include: page tracking hits, event tracking hits, ecommerce tracking hits, social interaction hits.

This made me believe that I should look at hits after all. The graph below shows a pattern that is very similar to the one of pageviews (no transactions for very few hits/pages, a correlation from x to y, and flat after that). Therefore, looking at the pageviews/hits ratio seems a good idea to me indeed! The second graph confirms the finding in the discussion topic (please notice that I have zoomed-in a lot make small amounts more visible)!


In [ ]:
h1 <- train %>% filter(hits <=200) %>% group_by(hits) %>%
summarize('sessions'= n(), 'transactions'= sum(transaction), 'pctTransactions'=round(x=((transactions/sessions)*100),digits=1)) %>%
    ggplot(aes(x=hits, y=pctTransactions)) +
    geom_bar(stat='identity', fill='blue') +
    geom_smooth()

train$ratioPageHits <- train$pageviews/train$hits

h2 <- train %>% filter(!is.na(pageviews)) %>%
    ggplot(aes(x=ratioPageHits, y=transactionRevenue)) +
    geom_line(stat='identity', col='blue') +
    coord_cartesian(y=c(0,100)) +
    scale_x_continuous(breaks=seq(0, 1, by=0.1))

grid.arrange(h1, h2)

## 2.7 Sessions, revenues and transactions by country
The US really dwarf all other countries with regards to the number of sessions and the total revenues.

In [ ]:
c1 <- plotSessionsFlip(train, country, 20)
c2 <- plotRevenueFlip(train, country, 20)
grid.arrange(c1, c2, nrow=1)

Only 10 countries have at least 100 sessions and at least 10 transaction, and only 4 of those countries have mean revenues per session of more than 1 USD. Mean revenues per transaction vary between 79 and 395 USD. However, we must realize that of the 11,515 transactions, almost all come from the US (10,953).

In [ ]:
train %>% group_by(country) %>%
summarize(sessions= n(), transactions=sum(transaction), totalRev=sum(transactionRevenue), sessionMean=totalRev/sessions, transactionMean=totalRev/transactions) %>%
filter(sessions>100 & transactions>10) %>% top_n(20, wt=sessionMean) %>% arrange(desc(sessionMean))

# 3. LightGBM Model
Please be aware that this is just a baseline model. It needs to be further optimized by Feature Engineering and Hyperparameter tuning. I have used some tips and tricks from kxx's kernel. Many thanks kxx!

I am using the last two months of the train set as my validation set.

In [ ]:
train <- train %>% select(-one_of(c("campaignCode", "transaction", "ratioPageHits")))

fea_uniq_values <- sapply(train, n_distinct)
(fea_del <- names(fea_uniq_values[fea_uniq_values == 1]))

train <- train %>% select(-one_of(fea_del))
test <- test %>% select(-one_of(fea_del))

In [ ]:
train$transactionRevenue <- train$transactionRevenue*1000000
train$pageviews[is.na(train$pageviews)] <- 0L
test$pageviews[is.na(test$pageviews)] <- 0L
idPageBounce <- test[, c("fullVisitorId", "pageviews", "bounces")]
trainIndex <- 1:nrow(train)
splitTrainValidDate <- ymd("20170601")

trainLabel <- log1p(train$transactionRevenue[train$date < splitTrainValidDate])
valLabel <- log1p(train$transactionRevenue[train$date >= splitTrainValidDate])
train$transactionRevenue <- NULL

#merging train and test do do some cleanup
all <- as.tibble(rbind(train, test))

is_na_val <- function(x) x %in% c("not available in demo dataset", "(not provided)",
                                  "(not set)", "<NA>", "unknown.unknown",  "(none)")

all <- all %>% mutate_all(funs(ifelse(is_na_val(.), NA, .)))

all <- all %>% mutate(isMobile = ifelse(isMobile, 1L, 0L),
            isTrueDirect = ifelse(isTrueDirect, 1L, 0L),
            adwordsClickInfo.isVideoAd = ifelse(!adwordsClickInfo.isVideoAd, 0L, 1L))

#There seems to be an issue with date and POSIXct after rbind
all$visitStartTime <- as.POSIXct(all$visitStartTime, tz="UTC", origin='1970-01-01')
all$date <- as.Date(all$date, origin='1970-01-01')

#new feature
all$sessionHourOfDay <- hour(all$visitStartTime)

all <- all %>%
select(-fullVisitorId, -visitId, -sessionId, -visitStartTime) %>% 
mutate_if(is.character, factor)

#keeping vector of categorical features for LGB
categorical_feature <- names(Filter(is.factor, all))

#Label encoding
all <- all %>% 
  mutate_if(is.factor, as.integer) %>% 
  glimpse()

In [ ]:
#splitting all into train, validation, and test set
dtrain <- all[trainIndex,]
dtest <- all[-trainIndex,]
dval <- dtrain[dtrain$date >= splitTrainValidDate,]
dtrain <- dtrain[dtrain$date < splitTrainValidDate,]

dtest$date <- NULL
dtrain$date <- NULL
dval$date <- NULL

In [ ]:
set.seed(123)

lgb.train = lgb.Dataset(data=as.matrix(dtrain),label=trainLabel, categorical_feature =categorical_feature)
lgb.valid = lgb.Dataset(data=as.matrix(dval),label=valLabel, categorical_feature =categorical_feature)

params <- list(objective="regression",
              metric="rmse",
              learning_rate=0.01)

lgb.model <- lgb.train(params = params,
                       data = lgb.train,
                       valids = list(val = lgb.valid),
                       learning_rate=0.01,
                       nrounds=1000,
                       verbose=1,
                       early_stopping_rounds=50,
                       eval_freq=100
                      )

lgb.model$best_iter
lgb.model$best_score 

In [ ]:
 tree_imp <- lgb.importance(lgb.model, percentage = TRUE)
 lgb.plot.importance(tree_imp, top_n = 50, measure = "Gain")

In [ ]:
pred <- predict(lgb.model, as.matrix(dtest)) %>% 
  as_tibble() %>% 
  set_names("y") %>% 
  mutate(y = expm1(y)) %>% 
  mutate(y = ifelse(y < 0, 0, y))
pred <- cbind(pred, idPageBounce)
pred <- pred %>%
  mutate(y = ifelse((pageviews<7|bounces==1), 0, y))
pred <- pred %>% select(-pageviews, -bounces) %>%
  group_by(fullVisitorId) %>% 
  summarise(y = log1p(sum(y)))

read_csv("../input/sample_submission.csv") %>%  
  left_join(pred, by = "fullVisitorId") %>% 
  mutate(PredictedLogRevenue = round(y, 5)) %>% 
  select(-y) %>% 
  write_csv(paste0("Lightgbm",round(lgb.model$best_score,5),".csv"))

The public score of this draft model is 1.46.

# Appendix: Screenshots of the app

The screenshots might be helpful to people to get some sort of domain knowledge. For now, I am including screenshots of all overview reports of the main menu.

## Main Screen

![](https://i.imgur.com/EUlHXDs.png)

## Real-time overview
The Real-Time Report allows you to monitor activity as it happens on your website – e.g. how many people are on your site right now, which pages or events they're interacting with, and which goal conversions have occurred.

![](https://i.imgur.com/5WadQz4.png)

## Audience

This report gives you a quick overview of the overall performance of the website – number of sessions,
users (both new and returning), page views, average session duration, bounce rate, new sessions.

I have entered the date range of train+test in Google Analytics. This leads to 1,8 million sessions. Below, you can also see that the total number of observations in the train and test sets is 1,7 million. I don't know yet where the discrepancies come from, but at least the number of sessions is in the same range indeed!

In [ ]:
nrow(train)+nrow(test)

![](https://i.imgur.com/WdooE8Q.png)

## Acquisition

This report gives you a quick overview of the top channels of your website traffic – direct traffic, organic
search, referral, social media, display advertising, email and paid search (i.e. AdWords) 

![](https://i.imgur.com/jVZ0tJ6.png)


## Behaviour
The Behavior overview report reveals what pages people visit on your website and what actions they take while visiting. It also provides information about the number of page views, bounce rate & exit rate.

![](https://i.imgur.com/fetx6qV.png)

## Conversions

![](https://i.imgur.com/0VpJu9i.png)